In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie
using OrderedCollections

# MakiePublication.jl has nice themes.

import NamedTrajectories as NT

Overview

1. Define ability to plot trajectories with series
2. Add a recipe so that we can send in custom kwargs (like transformation)
3. Add a plot function similar to our previous that creates a figure
4. Wrap the plot function in some way to allow for theme usage

In [ ]:
traj = rand(NT.NamedTrajectory, 10);

- We can use convert arguments to get access to plot and series commands
- n.b. Series is not in MakieCore; I'm unsure how this will impact us.
- We cannot use transformation as an argument because it conflicts with Makie's defaults.

In [ ]:
function convert_arguments(
    P::Type{<:Makie.Series}, 
    traj::NT.NamedTrajectory,
    name::Symbol;
    transform::Union{Function, Nothing}=nothing
)
    if !isnothing(transform)
        transform_data = try
            stack(transform.(eachcol(traj[name])))
        catch
            throw(ArgumentError("transform transformation of $(name) failed."))
        end
    else
        transform_data = traj[name]
    end

    return convert_arguments(P, NT.get_times(traj), transform_data)
end

MakieCore.used_attributes(::Type{<:Makie.Series}, ::NT.NamedTrajectory, ::Symbol) = (:transform,)
MakieCore.plottype(::NT.NamedTrajectory, ::Symbol) = Makie.Series

In [ ]:
f = Figure()
ax = Axis(f[1,1])

labels = ["x$i" for i in 1:size(traj.x, 1)]

p = plot!(
    ax, traj, :x, 
    transform=x -> x .^ 2, 
    labels=labels, color=:seaborn_colorblind, marker=:circle
)
Legend(f[1,2], ax)
f

- Name plot will have some default settings.

In [ ]:
# Add any desired series attributes here
@recipe(NamedPlot, traj, input_name, output_name, transformation) do scene
    Attributes(
        # color=:glasbey_bw_minc_20_n256,
        color=:seaborn_colorblind,
        linestyle=theme(scene, :linestyle),
        linewidth=theme(scene, :linewidth),
        marker=:circle,
        markersize=theme(scene, :markersize),
        merge=false,
    )
end

# Adds the ability to recall plot labels for a legend (extract series subplots)
Makie.get_plots(P::NamedPlot) = Makie.get_plots(P.plots[1])

# Plot existing component
function Makie.plot!(
    P::NamedPlot{<:Tuple{<:NT.NamedTrajectory, Symbol}};
    kwargs...
)
    lift(P[:traj], P[:input_name]) do traj, name

        if P[:merge][]
            labels = fill("$name", length(traj.components[name]))
        else
            labels = ["$(name) $(i)" for i in eachindex(traj.components[name])]
        end

        P = plot!(
            P, traj, name;
            labels=labels,
            color = P[:color],
            linestyle = P[:linestyle],
            linewidth = P[:linewidth],
            marker = P[:marker],
            markersize = P[:markersize],
        )
        
    end
    return P
end

# Plot transformed component
function Makie.plot!(
    P::NamedPlot{<:Tuple{<:NT.NamedTrajectory, Symbol, Union{Nothing, Symbol, String}, Function}};
    kwargs...
)
    lift(P[:traj], P[:input_name], P[:output_name], P[:transformation]) do traj, input, output, transform

        if isnothing(output)
            output = "T($input)"
        end

        if P[:merge][]
            labels = fill("$output", length(traj.components[input]))
        else
            labels = ["$(output) $(i)" for i in eachindex(traj.components[input])]
        end

        P = plot!(
            P, traj, input;
            transform=transform,
            labels=labels,
            color = P[:color],
            linestyle = P[:linestyle],
            linewidth = P[:linewidth],
            marker = P[:marker],
            markersize = P[:markersize],
        )
        
    end
    return P
end

In [ ]:
f = Figure()
ax = Axis(f[1,1])
p = namedplot!(ax, traj, :x)
Legend(f[1,2], ax)
f

In [ ]:
# Ability to apply transformations and merge names
f = Figure()
ax = Axis(f[1,1])
p = namedplot!(ax, traj, :x, "y", x -> x .^ 2, linewidth=3, marker=:circle, merge=true)
Legend(f[1,2], ax, merge=true)

ax = Axis(f[2,1])
p = namedplot!(ax, traj, :x, nothing, x -> x .^ 2, linewidth=3, marker=:circle)
Legend(f[2,2], ax)
f

- Plot trajectory will build the figure and allow use of themes

Notes:
- I've temporarily removed the vector of transformations (for each column) for simplicity. 
- What to do about ylims?
- I also removed the latex parsing (that's probably a theme variable, one of the default themes is LaTeX). Need to review what visual features should move to theme, and what should stay here.

In [ ]:
function plot_trajectory(
    traj::NT.NamedTrajectory,
    names::Union{AbstractVector{Symbol}, Tuple{Vararg{Symbol}}}=traj.names;

    # ---------------------------------------------------------------------------
    # component specification keyword arguments
    # ---------------------------------------------------------------------------

    # whether or not to plot the timestep componenent
    ignore_timestep::Bool=true,
    
    # whether or not to include unique labels for components
    merge_labels::Union{Bool, AbstractVector{Bool}} = false,

    # ---------------------------------------------------------------------------
    # transformation keyword arguments
    # ---------------------------------------------------------------------------

    # transformations
    transformations::OrderedDict{Symbol, <:Function} = OrderedDict{Symbol, Function}(),

    # labels for transformed components
    transformation_labels::AbstractVector{<:Union{Nothing, String}} = fill(nothing, length(transformations)),
    
    # whether or not to include unique labels for transformed components
    merge_transformation_labels::Union{Bool, AbstractVector{Bool}} = false,
    
    # ---------------------------------------------------------------------------
    # figure and axis keyword arguments
    # ---------------------------------------------------------------------------
    
    fig_size::Tuple{Int, Int} = (1200, 800),
    titlesize::Int=16,
    xlabelsize::Int=16,

    # ---------------------------------------------------------------------------
    # plot keyword arguments (for all names)
    # ---------------------------------------------------------------------------
    kwargs...
)

    # parse arguments
    if names isa Symbol
        names = [names]
    end

    if merge_labels isa Bool
        merge_labels = fill(merge_labels, length(names))
    end

    if merge_transformation_labels isa Bool
        merge_transformation_labels = fill(merge_transformation_labels, length(transformations))
    end

    @assert length(merge_transformation_labels) == length(transformation_labels) == length(transformations)

    # create figure
    fig = Figure(size=fig_size)

    # Default components
    # ------------------
    for (i, name) in enumerate(names)
        ax = Axis(
            fig[i, 1],
            title = i == 1 ? "Named Trajectory" : "",
            titlealign = :left,
            titlesize = titlesize,
            xticklabelsvisible = i == length(names),
            xtickalign=1,
            xlabel = i == length(names) ? "time" : "",
            xlabelsize = xlabelsize,
        )
        merge = merge_labels[i]
        namedplot!(ax, traj, name, merge=merge; kwargs...)
        Legend(fig[i, 2], ax, merge=merge)
    end

    for i in 1:length(names) - 1
        rowgap!(fig.layout, i, 0.0)
    end


    # Transformations
    # ---------------
    offset = length(names)

    for (i, (input, transform)) in enumerate(transformations)
        ax = Axis(
            fig[offset + i, 1],
            title = i == 1 ? "Transformations" : "",
            titlealign = :left,
            titlesize = titlesize,
            xticklabelsvisible = i == length(transformations),
            xtickalign = 1,
            xlabel = i == length(transformations) ? "time" : "",
            xlabelsize = xlabelsize,
        )
        output = transformation_labels[i]
        merge = merge_transformation_labels[i]
        namedplot!(ax, traj, input, output, transform, merge=merge; kwargs...)
        Legend(fig[offset + i, 2], ax, merge=merge)
    end

    for i in 1:length(transformations)-1
        rowgap!(fig.layout, offset + i, 0.0)
    end

    fig
end

function plot_trajectory(
    theme::MakieCore.Theme,
    args...;
    kwargs...
)
    with_theme(theme) do
        plot_trajectory(args...; kwargs...)
    end
end

In [ ]:
plot_trajectory(
    traj, 
    [:x, :u],
    merge_labels=[true, false],
    transformations=OrderedDict(:x => x -> [x[1] ^6],), 
    merge_transformation_labels=true,
)

Test passing in series args

In [ ]:
plot_trajectory(
    traj, 
    [:x, :u],
    merge_labels=[true, false],
    transformations=OrderedDict(:x => x -> [x[1] ^6],), 
    merge_transformation_labels=true,
    linewidth=5,
    color=:glasbey_bw_minc_20_n256,
)

Test setting a theme.

In [ ]:
plot_trajectory(theme_latexfonts(), traj, merge_labels=true)